<a id='rdw_top'>

# Add extra information from RDW open data

Query to the open data dataset of the RDW.


1. <a href="#rdw_registrations">Registration numbers</a>  
    Apis with registration as key
2. <a href="#rdw_confcodes">Conformity codes</a>  
    Cars get a conformity code when certified.
3. <a href="#rdw_other_apis">Other APIs</a>  
    It may take a while (10 min) to query all conformity codes individually.
4. <a href="#rdw_ovi">Closed data</a>  
    Get closed data from RDW website. This also takes a while because of time out enforced by website. Use config to disable.
5. <a href="#rdw_merge">Merge results</a>  
    Combine all dataframes and save

- - - - 

### User variables


In [ ]:
import drz_config
cfg = drz_config.read_config()
DATE = cfg['DATE']
VERBOSE = cfg['VERBOSE']
OPBOD = cfg['OPBOD']
CLOSEDDATA = cfg['CLOSEDDATA']
SKIPSAVE = cfg['SKIPSAVE']

if VERBOSE > 0:
    print(cfg)

### Modules and functions

In [ ]:
import pandas as pd
import re 
# to keep api key hidden import this from sub dir
import assets.hidden_api_keys as hidden_api_keys
from time import sleep
import urllib

# base url
apiurl = 'https://opendata.rdw.nl/resource/m9d7-ebf2.json?$$app_token=' + hidden_api_keys.socrata_apptoken + '&'

def get_json_from_api(url,reg,c=0):
    
    '''Get json object from api'''
    
    import time

    c+=1
    try:
        df=pd.read_json(url + 'kenteken=' + reg.replace('-','').upper()).to_dict()
    except:
        if c > 10:
            print(url,reg)
            raise 
        else:
            print('pause 2 sec and try again!')
            time.sleep(2)
            df = get_json_from_api(url,reg,c)
    
    return df
    
# get_json_from_api(apiurl,'61-sf-FG')

### Load auction results

In [ ]:
if OPBOD:
    file_name = '../../../python-nb/data/drz-data-opbod-{}.pkl'.format(DATE)
else:
    file_name = '../data/drz-data-{}.pkl'.format(DATE)
print(file_name)
drz = pd.read_pickle(file_name)

### Collect registrations

In [ ]:
# see what lots have a Dutch registration (license number).
hasReg = (~drz.Reg.isnull()) & (drz.Reg != 'onbekend') & (drz.Reg != '') & (~drz.LotType.isin([
    'Vaartuig',
    'Jetski',
    'Sloep',
    'Speedboot',
    'Vaartuig (Type onbekend)',
    'Motorvaartuig met opbouw (Pleziervaartuig)',
    'Aanhangwagen',
]))

print('nr. of registrations:',sum(hasReg))

# make a copy and add info
rdw = drz.copy()


In [ ]:
def get_query_url(api_url, keys, token=hidden_api_keys.socrata_apptoken, query_field='kenteken'):
    '''construct query url'''

    import urllib

    # First part
    q = api_url + '?$$app_token=' + token + '&'

    # convert list to string
    id_list = ''.join(["'{}', ".format(k) for k in keys])
    id_list = id_list[0:-2] # remove trailing ', '
    
    # add escaped soql
    soql = '$where=' + urllib.parse.quote(query_field + ' in (' + id_list + ')')
    q += soql
    
    # See if field is available
    accepted_fields = eval(urllib.request.urlopen(api_url).headers.get('X-SODA2-Fields'))
    if query_field not in accepted_fields:
        raise ValueError(f'<{query_field}> not allowed as SODA2 field in {api_url}.\n\tAccepted are: ' + ', '.join(accepted_fields))
    
    return q

Create list of dataframes with different api results

In [ ]:
# empty dictionary
rdw_per_reg = dict()

# first element of dict is registrations
key = 'registations'
rdw_per_reg[key] = rdw.loc[hasReg,['Reg']] # copy from df
rdw_per_reg[key]['kenteken'] = rdw_per_reg[key].Reg.apply(lambda r: r.replace('-','').upper())
rdw_per_reg[key].index.name = 'lot_index'
rdw_per_reg[key] = rdw_per_reg[key].reset_index().set_index('kenteken')
display(rdw_per_reg[key])


<H1><a href="#rdw_top">^</a></H1><a id='rdw_registrations'>

# Main apis

The main api: `api_gekentekende_voertuigen` points to subsequent apis.

In [ ]:
# Assess these registrations
regs = rdw_per_reg['registations'].Reg.values
regs = [r.replace('-','').upper() for r in regs]
print(len(regs),'registrations in this set')

# Do main api first to get other possible apis
api_name = 'api_gekentekende_voertuigen'
api_url = 'https://opendata.rdw.nl/resource/m9d7-ebf2.json'

# Query api
key = re.sub('^api_','', api_name) # store with this key
q = get_query_url(api_url,regs)
rdw_per_reg[key] = pd.read_json(q)
rdw_per_reg[key].set_index('kenteken', inplace=True)

print(api_name, end=': ')
display(rdw_per_reg[key])

# Query other available apis
for api_name in [c for c in rdw_per_reg['gekentekende_voertuigen'].columns if c.startswith('api')]:
    print(api_name, end=': ')
    key = re.sub('^api_','',api_name)
    for api_url in rdw_per_reg['gekentekende_voertuigen'][api_name].unique():
        print(api_url)
        # query the web
        q = get_query_url(api_url,regs)
        df0 = pd.read_json(q)
        # name of index
        df0.columns.name = api_name

        # query should return 'kenteken', make it the index
        if df0.shape[0] != 0:
            df0.set_index('kenteken', inplace=True)
            
        # Some apis return multiple values. Pivot around index number ("volgnummer")
        if api_name == 'api_gekentekende_voertuigen_assen':
            df0 = pd.pivot(df0, columns='as_nummer')

        elif api_name == 'api_gekentekende_voertuigen_brandstof':
            df0 = pd.pivot(df0, columns='brandstof_volgnummer')

        elif api_name == 'api_gekentekende_voertuigen_carrosserie':
            df0 = pd.pivot(df0, columns='carrosserie_volgnummer')

        elif api_name == 'api_gekentekende_voertuigen_carrosserie_specifiek':
            df0 = pd.pivot(df0, columns='carrosserie_voertuig_nummer_code_volgnummer')

        # squeeze multi index
        one_level = [
            re.sub('^api_gekentekende_voertuigen_','',api_name) + '_' + '_'.join(
                [str(c) if type(c)==int else c for c in l]
            ) for l in df0.columns
        ]
        df0.columns = one_level

        # add to list
        display(df0.tail(3))
        print(df0.shape)
        rdw_per_reg[key]=df0

# Merge dataframes from different apis
df_regs = pd.concat(rdw_per_reg.values(), axis='columns', sort=False)
# add timestamp
df_regs['TimeStamp'] = pd.to_datetime('now').strftime('%Y%m%d')
# set lot id as index
df_regs.index.name = 'kenteken'
df_regs = df_regs.reset_index().set_index('lot_index')
display(df_regs)

<H1><a href="#rdw_top">^</a></H1><a id='rdw_confcodes'>

# Conformity codes

In [ ]:
# empty dict
rdw_per_confcode = dict()
# Conformity codes have sub-divisions. Four fields make a super key
key = 'conformity_codes'
rdw_per_confcode[key] = rdw_per_reg['gekentekende_voertuigen'][[
    'typegoedkeuringsnummer', 
    'uitvoering', 
    'variant', 
    'volgnummer_wijziging_eu_typegoedkeuring'
]].dropna().drop_duplicates()
rdw_per_confcode[key].reset_index(drop=True, inplace=True)
print(len(rdw_per_confcode[key]),'conformity codes in this set')

display(rdw_per_confcode[key])
print('count unique')
display(\
    rdw_per_reg['gekentekende_voertuigen'].loc[:, rdw_per_confcode[key].columns]\
    .reset_index()\
    .groupby([rdw_per_confcode[key].columns[0]])\
    .nunique()\
    .replace(1,pd.np.NaN)\
    .dropna(how='all')\
    .fillna(1)\
    .astype(int)\
    .sort_values(by='kenteken', ascending=False)
)

In [ ]:
# do conformity api and again get other possible apis
api_name = 'api_eeg_voertuigtypegoedkeuring'
api_url = 'https://opendata.rdw.nl/resource/55kv-xf7m.json'

# Query api
key = re.sub('^api_','', api_name) # store with this key
q = get_query_url(
    api_url, 
    rdw_per_confcode['conformity_codes'].typegoedkeuringsnummer.unique(), # Use long (year with century) version of conformity code
    query_field='typegoedkeuringsnummer'
)
rdw_per_confcode[key] = pd.read_json(q)
# more than one conformity code?
assert not (rdw_per_confcode[key].groupby('typegoedkeuringsnummer')['typegoedkeuringsnummer'].count() > 1).any()
rdw_per_confcode[key].set_index('typegoedkeuringsnummer', inplace=True)

# add slightly different keys (year has no century)
rdw_per_confcode['conformity_codes'] = rdw_per_confcode['conformity_codes'].merge(
    rdw_per_confcode['eeg_voertuigtypegoedkeuring'].eu_type_goedkeuringssleutel, 
    how='left', 
    left_on='typegoedkeuringsnummer', 
    right_index=True
)

print(api_name, end=': ')
display(rdw_per_confcode[key])




<H1><a href="#rdw_top">^</a></H1><a id='rdw_other_apis'>

# Other apis

In [ ]:
# query other available apis
api_list = [c for c in rdw_per_confcode['eeg_voertuigtypegoedkeuring'].columns if c.startswith('api')]
for api_name in api_list:
    print(api_name, end=''.join(['|' if api_name==list_element else '.'  for list_element in api_list]))
    key = re.sub('^api_','',api_name)
    for api_url in rdw_per_confcode['eeg_voertuigtypegoedkeuring'][api_name].unique():
        
        # reformat url
        M=re.search('https://opendata.rdw.nl/.*/([a-z0-9]{4}-[a-z0-9]{4})$', api_url)
        api_url = 'https://opendata.rdw.nl/resource/{}.json'.format(M[1])
        print(api_url)
       
        # query the web
        # unfortunately this needs to be done one by one, because conformity code is not unique                   
        df0 = pd.DataFrame()
        for ix, row in rdw_per_confcode['conformity_codes'].iterrows():
            q = api_url
            q += '?$$app_token=' + hidden_api_keys.socrata_apptoken 
            q += '&{}=\'{}\''.format('eu_type_goedkeuringssleutel', urllib.parse.quote(row.eu_type_goedkeuringssleutel))
            q += '&{}=\'{}\''.format('eeg_uitvoeringscode', urllib.parse.quote(row.uitvoering))
            q += '&{}=\'{}\''.format('eeg_variantcode', urllib.parse.quote(row.variant))
            q += '&{}={:.0f}'.format('uitvoering_wijzigingsnummer', row.volgnummer_wijziging_eu_typegoedkeuring)
                
            n_try = 0
            OK = False
            while (n_try < 10) & (not OK):
                try:
                    res = pd.read_json(q)
                    OK = True
                    n_try = 0
                except:
                    n_try +=1
                    sleep(10)
                if n_try == 10:
                    raise

            if len(res) == 0:
                continue

            # matching data type with 'codes' for merging
            res.eu_type_goedkeuringssleutel = res.eu_type_goedkeuringssleutel.astype(str)
            res.eeg_uitvoeringscode = res.eeg_uitvoeringscode.astype(str)
            res.eeg_variantcode = res.eeg_variantcode.astype(str)
            res.uitvoering_wijzigingsnummer = res.uitvoering_wijzigingsnummer.astype(float)
            res.set_index([
                'eu_type_goedkeuringssleutel', 
                'eeg_uitvoeringscode', 
                'eeg_variantcode', 
                'uitvoering_wijzigingsnummer'
            ], inplace=True)
                
                
            # Api specific modifiers
            if api_name == 'api_as_gegevens_eeg_uitvoering':               
                piv = res.pivot(columns='asnummer')
                one_level = [
                            re.sub('^api_','', re.sub('_eeg_uitvoering$','',api_name)) + '_' + '_'.join(
                                [str(c) if type(c)==int else c for c in l]
                            ) for l in piv.columns
                        ]
                piv.columns = one_level
                df0 = pd.concat([df0, piv], sort=False)                
            elif api_name == 'api_handelsbenaming_uitvoering':                
                piv = res.pivot(columns='volgnummer')
                one_level = [
                            re.sub('^api_','', re.sub('_eeg_uitvoering$','',api_name)) + '_' + '_'.join(
                                [str(c) if type(c)==int else c for c in l]
                            ) for l in piv.columns
                        ]
                piv.columns = one_level
                df0 = pd.concat([df0, piv], sort=False)
            elif api_name == 'api_carrosserie_uitvoering':                
                piv = res.pivot(columns='carrosserie_volgnummer')
                one_level = [
                            re.sub('^api_','', re.sub('_eeg_uitvoering$','',api_name)) + '_' + '_'.join(
                                [str(c) if type(c)==int else c for c in l]
                            ) for l in piv.columns
                        ]
                piv.columns = one_level
                df0 = pd.concat([df0, piv], sort=False)
            elif api_name == 'api_plaatsaanduiding_uitvoering':
                piv = res.pivot(columns='plaats_aanduiding_volgnummer')
                one_level = [
                            re.sub('^api_','', re.sub('_eeg_uitvoering$','',api_name)) + '_' + '_'.join(
                                [str(c) if type(c)==int else c for c in l]
                            ) for l in piv.columns
                        ]
                piv.columns = one_level
                df0 = pd.concat([df0, piv], sort=False)            
            elif api_name == 'api_uitvoeringverbruik_per_uitgave':
                piv = res.pivot(columns='uitvgavenummer_verbruikboek')
                one_level = [
                            re.sub('^api_','', re.sub('_eeg_uitvoering$','',api_name)) + '_' + '_'.join(
                                [str(c) if type(c)==int else c for c in l]
                            ) for l in piv.columns
                        ]
                piv.columns = one_level
                df0 = pd.concat([df0, piv], sort=False)
            elif api_name == 'api_motor_uitvoering':
                piv = res.pivot(columns='volgnummer')
                one_level = [
                            re.sub('^api_','', re.sub('_eeg_uitvoering$','',api_name)) + '_' + '_'.join(
                                [str(c) if type(c)==int else c for c in l]
                            ) for l in piv.columns
                        ]
                piv.columns = one_level
                df0 = pd.concat([df0, piv], sort=False)
            elif api_name == 'api_versnellingsbak_uitvoering':
                piv = res.pivot(columns='volgnummer')
                one_level = [
                            re.sub('^api_','', re.sub('_eeg_uitvoering$','',api_name)) + '_' + '_'.join(
                                [str(c) if type(c)==int else c for c in l]
                            ) for l in piv.columns
                        ]
                piv.columns = one_level
                df0 = pd.concat([df0, piv], sort=False)
            elif api_name == 'api_motor_uitvoering_brandstof':
                piv = pd.pivot_table(res, index=res.index.names, columns=['volgnummer', 'brandstof_volgnummer'])
                one_level = [
                            re.sub('^api_','', re.sub('_eeg_uitvoering$','',api_name)) + '_' + '_'.join(
                                [str(c) if type(c)==int else c for c in l]
                            ) for l in piv.columns
                        ]
                piv.columns = one_level
                df0 = pd.concat([df0, piv], sort=False)                
            elif api_name == 'api_merk_uitvoering_toegestaan':
                piv = pd.pivot_table(res, index=res.index.names, values=res.columns, aggfunc=list)
                df0 = pd.concat([df0, piv], sort=False)
            else:
                assert res.shape[0] == 1 # Nothing to pivot on
                columns = ['{}_{}'.format(re.sub('^api_','', re.sub('_eeg_uitvoering$','',api_name)), c) for c in res.columns] 
                res.columns = columns
                df0 = pd.concat([df0, res], sort=False)
                
            print('.', end='')
        if len(df0)==0:
            print('No results for this api')
            continue


        # add to dict
        rdw_per_confcode[key] = df0
        display(rdw_per_confcode[key].tail())
        print(rdw_per_confcode[key].shape)


<H1><a href="#rdw_top">^</a></H1><a id='rdw_ovi'>

# Closed data from rdw (OVI)
Optionally get data from rdw website

In [ ]:
def get_closed_data(reg, fields = ['Eigenaren', 'WachtenOpKeuring']):
    web_url = f'https://ovi.rdw.nl/?kenteken={reg}'

    import requests
    import codecs
    from lxml import html, etree
    import re
    from time import sleep

    page = requests.get(web_url)
    DecodeType = re.findall('charset=(.*)$', page.headers["Content-type"])[0]
    htmlstring = codecs.decode(page.content, DecodeType)

    # Convert string to tree object
    tree = html.fromstring(htmlstring)
    
    out = dict()
    TRY_AGAIN = False
    for fld in fields:
        txt = tree.xpath(f'//*[@id="{fld}"]/text()')
        if len(txt) != 1:
            # No result. Probably limit request
            TRY_AGAIN = True
            break
        out[fld] = txt[0]
    if TRY_AGAIN:
        # pause
        sleep(1)
        print('x', end='')
        out = get_closed_data(reg, fields)
    
    return out
    

In [ ]:
if CLOSEDDATA:
    print('x: timeout\n')
    data = {reg: get_closed_data(reg) for reg in df_regs.kenteken}
    rdw_closed = pd.DataFrame.from_dict(data=data, orient='index')
else:
    rdw_closed = None

rdw_closed

<H1><a href="#rdw_top">^</a></H1><a id='rdw_merge'>

# Merge datasets
- Merge dataframes from conformity codes apis
- Merge with registration results
- Merge with auction results

In [ ]:
# first merge first two results
tmp = rdw_per_confcode['conformity_codes'].merge(rdw_per_confcode['eeg_voertuigtypegoedkeuring'], how='left', 
                                              left_on='typegoedkeuringsnummer',
                                              right_index=True
                                             )

tmp.rename(columns={
    'eu_type_goedkeuringssleutel_x': 'eu_type_goedkeuringssleutel',
    'uitvoering': 'eeg_uitvoeringscode',
    'variant': 'eeg_variantcode',
    'volgnummer_wijziging_eu_typegoedkeuring': 'uitvoering_wijzigingsnummer',
}, inplace=True)
tmp.set_index(['eu_type_goedkeuringssleutel', 'eeg_uitvoeringscode', 'eeg_variantcode', 'uitvoering_wijzigingsnummer'], inplace=True)

# merge with subsequent api results
df_confcodes = pd.concat([tmp] + list(rdw_per_confcode.values())[2:], axis='columns', sort=False)
# add timestamp
df_confcodes['TimeStamp'] = pd.to_datetime('now').strftime('%Y%m%d')

display(df_confcodes.tail())
print(df_confcodes.shape)

In [ ]:
# Merge confirmation codes with registrations
df_regs.index.name = 'lot_index'
df = df_regs.reset_index().merge(df_confcodes.reset_index(), how='left',
                   left_on=['typegoedkeuringsnummer', 'uitvoering', 'variant', 'volgnummer_wijziging_eu_typegoedkeuring'],
                   right_on=['typegoedkeuringsnummer', 'eeg_uitvoeringscode', 'eeg_variantcode', 'uitvoering_wijzigingsnummer'],
).set_index('lot_index')
display(df.tail(10))
print(df.shape)
assert all(df.columns.value_counts() == 1)

In [ ]:
# Merge closed source data with registrations

rdw_closed.index.name='kenteken' 
rdw_closed['ovi_private_owners'] = rdw_closed.Eigenaren.str.split('/').apply(lambda x:int(x[0].strip()))
rdw_closed['ovi_company_owner'] = rdw_closed.Eigenaren.str.split('/').apply(lambda x:int(x[1].strip()))
rdw_closed['ovi_owners'] = rdw_closed['ovi_private_owners'] + rdw_closed['ovi_company_owner']
rdw_closed['ovi_under_survey'] = rdw_closed.WachtenOpKeuring.apply(lambda x: {'Ja': True, 'Nee': False}[x])

df = df.merge(rdw_closed.loc[:,[c for c in rdw_closed.columns if c.startswith('ovi_')]],
              how='left', left_on='kenteken', right_index=True)

### Merge rdw and drz

In [ ]:
rdw = pd.concat([rdw, df.add_prefix('rdw_')], axis='columns', sort=False)
# There should be no duplicates in column names
assert all(rdw.columns.value_counts() == 1)
# indices should match
assert rdw.index.isin(drz.index).all() & drz.index.isin(rdw.index).all()

# Saving

In [ ]:
if OPBOD:
    file_name = f'../../../python-nb/data/rdw-data-opbod-{DATE}.pkl'
else:
    file_name = f'../data/rdw-data-{DATE}.pkl'

if (SKIPSAVE==False) and (not(os.path.isfile(file_name))):
    print(file_name)
    out.to_pickle(file_name)
else:
    print(f'Skip. {file_name} exists or saving is disabled in settings.')

# Next: download images (or parallel)

Because images might be taken down from the drz site, it is advisable to run the notebook that downloads images soon.